In [1]:
#!/usr/bin python2.7

from astropy.io       import fits
from astropy.table    import Table
from astropy.time     import Time
from datetime         import date
from datetime         import datetime
import ephem
import fileinput
import getpass
import math
import matplotlib.pyplot                   as plt
import matplotlib                          as mpl
import numpy                               as np
from numpy            import linalg        as LA
import os
import pyfits
import scipy.optimize                      as optimization
from scipy            import integrate
from scipy            import interpolate
from scipy.optimize   import curve_fit
import subprocess
from subprocess       import Popen, PIPE
import urllib2
from work_module      import calculate
from work_module      import detector
from work_module      import readfile
from work_module      import writefile
calc = calculate()
det = detector()
rf = readfile()
wf = writefile()

In [69]:
### Day and Detector settings ###
day = 160913
day2 = 101225
day3 = 111209
day4 = 121027
day5 = 130925
day6 = 141010
day7 = 150926
detector = 'n5'
data_type = 'ctime'
echan = 2



In [ ]:
### Altitude ###

pos_data = rf.poshist(day)
sat_time = pos_data[0]
sat_pos = pos_data[1]
sat_lat = pos_data[2]
sat_lon = pos_data[3]
sat_q = pos_data[4]

sat_dist = LA.norm(sat_pos, axis=0) #from the Earth's center

ell_a = 6378137
ell_b = 6356752.3142
sat_lat_rad = sat_lat*2*math.pi/360.
earth_radius_sat = ell_a * ell_b / ((ell_a**2 * np.sin(sat_lat_rad)**2 + ell_b**2 * np.cos(sat_lat_rad)**2)**0.5)

altitude = (sat_dist - earth_radius_sat)/1000

print np.mean(altitude)

plot_time_sat_date = calc.met_to_date(sat_time)[0]
plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24

plt.plot(plot_time_sat, altitude, 'r-', label = 'Altitude', linewidth =2)

plt.grid()
#plt.legend()

plt.xlim([-0.5, 24.5])
plt.ylim([500, 560])

plt.title('Altitude of the Fermi Satellite on September 13th 2016', fontsize=40)
plt.xlabel('Daytime in 24h', fontsize=40)
plt.ylabel('Altitude in km', fontsize=40)

plt.tick_params(axis='both', which='major', labelsize=30)
#plt.tick_params(axis='both', which='minor', labelsize=8)

plt.show()

In [ ]:
### Longitude + Latitude ###

pos_data = rf.poshist(day)
sat_time = pos_data[0]
sat_pos = pos_data[1]
sat_lat = pos_data[2]
sat_lon = pos_data[3]
sat_q = pos_data[4]

times = []
for i in range(1, len(sat_time)):
    if np.logical_and(sat_lon[i-1] > 350, sat_lon[i] < 10):
        times.append(sat_time[i])
    #elif np.logical_and(sat_pos[0][i-1] < 0, sat_pos[0][i] > 0):
    #    print '2 ', sat_time[i]
#print sat_time[np.where(np.fabs(sat_pos[0]) < 100)]

times = np.array(times)
print (times[-1] - times[0])/((len(times)-1)*60) #Earth-orbit
print np.max(sat_lat) #inclination

In [ ]:
### J2000-Orbit ###

pos_data = rf.poshist(day)
sat_time = pos_data[0]
sat_pos = pos_data[1]
sat_lat = pos_data[2]
sat_lon = pos_data[3]
sat_q = pos_data[4]

times = []
lats = []
for i in range(1, len(sat_time)):
    if np.logical_and(sat_pos[0][i-1] > 0, sat_pos[0][i] < 0):
        times.append(sat_time[i])
        lats.append(sat_lat[i])
    #elif np.logical_and(sat_pos[0][i-1] < 0, sat_pos[0][i] > 0):
    #    print '2 ', sat_time[i]
#print sat_time[np.where(np.fabs(sat_pos[0]) < 100)]

times = np.array(times)
lats = np.array(lats)
#for i in range(1, len(times)):
#    print (times[i] - times[i-1])/60
print (times[-1] - times[0])/((len(times)-1)*60) #J2000-orbit
print lats

In [ ]:
### precession ###

month1 = np.arange(31) + 160701
month2 = np.arange(31) + 160801
month3 = np.arange(19) + 160901
days = np.append(month1, np.append(month2, month3))

times = []
lats = []

#days = np.delete(days, [13])
#print days

for day in days:
    #print day
    pos_data = rf.poshist(day)
    sat_time = pos_data[0]
    sat_pos = pos_data[1]
    sat_lat = pos_data[2]
    sat_lon = pos_data[3]
    sat_q = pos_data[4]
    
    for i in range(1, len(sat_time)):
        if np.logical_and(sat_pos[0][i-1] > 0, sat_pos[0][i] < 0):
            times.append(sat_time[i])
            lats.append(sat_lat[i])

lats = np.array(lats)
times = np.array(times)

precession = times[np.where(np.logical_and(lats >= -0.1, lats <= 0.1))]

precession_time = (precession[2] - precession[0])/3600/24
print precession_time

In [ ]:
print np.where(np.logical_and(lats >= -0.3, lats <= 0.3))
print lats[np.where(np.logical_and(lats >= -0.3, lats <= 0.3))]
print times[np.where(np.logical_and(lats >= -0.1, lats <= 0.1))]
precession = times[np.where(np.logical_and(lats >= -0.1, lats <= 0.1))]

#print precession
#precession = np.array(precession)
precession_time = (precession[2] - precession[0])/3600/24
print precession_time

In [ ]:
### Fermi-Satellite Orbital-behaviour ###

days = np.array([101225, 111209, 121027, 130925, 141010, 150926, 160913])
years = np.array([2010, 2011, 2012, 2013, 2014, 2015, 2016])
str_years = np.array(['2010', '2011', '2012', '2013', '2014', '2015', '2016'])
altitude = np.array([550.645, 548.651, 546.451, 544.102, 539.703, 535.571, 533.763])
j2000_orb = np.array([95.575, 95.532, 95.489, 95.458, 95.346, 95.268, 95.250])
earth_orb = np.array([102.44, 102.39, 102.30, 102.28, 102.10, 102.03, 102.05])
inclination = np.array([25.55, 25.57, 25.55, 25.50, 25.55, 25.53, 25.66])

plt.plot(years, altitude, 'r-', label = 'Altitude in km', linewidth =2)
#plt.plot(years, inclination, 'y-', label = 'Inclination in degrees')
#plt.plot(years, j2000_orb, 'b-', label = 'J2000 orbital period in min')
#plt.plot(years, earth_orb, 'g-', label = 'Geographic orbital period in min')

plt.grid()
#plt.legend()

plt.xlim([2010, 2016])
plt.ylim([519, 560])

plt.xticks(years, str_years)
plt.title('Altitude of the Fermi Satellite', fontsize=40)
plt.xlabel('Year', fontsize=40)
plt.ylabel('Altitude in km', fontsize=40)

plt.tick_params(axis='both', which='major', labelsize=30)
#plt.tick_params(axis='both', which='minor', labelsize=8)

plt.show()

In [ ]:
### Time conversion tests ###

met = 448588740.740073
day = 150321

data = calc.met_to_date(met)
mjdutc = data[0]
mjdtt = data[1]
isot = data[2]
date = data[3]
decimal = data[4]

print mjdutc
print mjdtt
print isot
print date
print decimal

sun = calc.sun_pos(day)

In [ ]:
### Angular calibration for ctime channels ###
##### WRONG!!!!WRONG!!!!WRONG!!!! #####

fitsname = 'peak_eff_area_angle_calib_GBM_all.fits'
user = getpass.getuser()
fits_path = '/home/' + user + '/Work/calibration/'
fitsfilepath = os.path.join(fits_path, fitsname)
fitsfile = fits.open(fitsfilepath, mode='update')
data = fitsfile[1].data
fitsfile.close()
x = data.field(0)
y1 = data.field(1)#for NaI (33 keV)
y2 = data.field(2)#for NaI (279 keV)
y3 = data.field(3)#for NaI (662 keV)
y4 = data.field(4)#for BGO (898 keV)
y5 = data.field(5)#for BGO (1836 keV)
        
ang_eff = []

echans_energy = np.array([8, 19, 38, 75, 198, 417, 762, 1492])

#ctime linear-interpolation factors
#y1_fac = np.array([1.2, 1.1, 1., 1., 1., 0., 0., 0.])
#y2_fac = np.array([0., 0., 5./279., 42./279., 165./279., 1., 0., 0.])
#y3_fac = np.array([0., 0., 0., 0., 0., 138./662., 1., 1.])

### Angepasst ###
#y1_fac = np.array([1.2, 1.08, 238./246., 196./246., 77./246., 0., 0., 0.])
#y2_fac = np.array([0., 0., 5./246., 40./246., 159./246., 230./383., 0., 0.])
#y3_fac = np.array([0., 0., 0., 0., 0., 133./383., .7, .5])

###Angepasst 2 ###
y1_fac = np.array([1.2, 1.08, 238./246., 196./246., 127./246., 0., 0., 0.])
y2_fac = np.array([0., 0., 5./246., 40./246., 109./246., 230./383., 0., 0.])
y3_fac = np.array([0., 0., 0., 0., 0., 133./383., .7, .5])

mpl.rcParams['axes.color_cycle'] = ['b', 'g', 'r', 'c', 'm', 'y', 'k', '0.5']

for i in range(0, len(echans_energy)):
    #resulting effective area curve
    y = y1_fac[i]*y1 + y2_fac[i]*y2 + y3_fac[i]*y3
    
    #normalize
    y = y
    
    plt.plot(x, y, label = 'Energy channel ' + str(i))

plt.plot(x, y1, 'k--', label = '33 keV')
plt.plot(x, y2, 'b--', label = '279 keV')
plt.plot(x, y3, 'g--', label = '662 keV')
plt.xlabel('Angle in degrees', fontsize=20)
plt.ylabel('Normalized photopeak effective area', fontsize=20)
plt.grid()
plt.legend()
plt.show()

In [ ]:
y_new = np.array([y1[90], y2[90], y3[90]])
x_new = np.array([33, 279, 662])

### Schnittpunkte ###
#y1_fac = np.array([1.2, 1.1, 241./246., 204./246., 81./246., 0., 0., 0.])
#y2_fac = np.array([0., 0., 5./246., 42./246., 165./246., 245./383., 0., 0.])
#y3_fac = np.array([0., 0., 0., 0., 0., 138./383., .7, .5])

### Angepasst ###
#y1_fac = np.array([1.2, 1.08, 238./246., 196./246., 77./246., 0., 0., 0.])
#y2_fac = np.array([0., 0., 5./246., 40./246., 159./246., 230./383., 0., 0.])
#y3_fac = np.array([0., 0., 0., 0., 0., 133./383., .7, .5])

###Angepasst 2 ###
y1_fac = np.array([1.2, 1.08, 238./246., 196./246., 127./246., 0., 0., 0.])
y2_fac = np.array([0., 0., 5./246., 40./246., 109./246., 230./383., 0., 0.])
y3_fac = np.array([0., 0., 0., 0., 0., 133./383., .7, .5])

y_echans = y1_fac*y1[90] + y2_fac*y2[90] + y3_fac*y3[90]

plt.plot(x_new, y_new/y1[90], 'k--')
plt.plot(echans_energy, y_echans/y1[90], 'g-')
plt.xlabel('Energy in keV', fontsize=20)
plt.ylabel('Normalized photopeak effective area maximum', fontsize=20)
plt.xlim([0, 1600])
plt.show()

In [ ]:
### Earth occultation vs. Earth angle comparison ###

#day = 150926
#detector = 'n5'
#echan = 2
#data_type = 'ctime'

year = int('20' + str(day)[0:2])

#get the iso-date-format from the day
date = datetime(year, int(str(day)[2:4]), int(str(day)[4:6]))

#get the ordinal indicator for the date
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])

#read the measurement data
ctime_data = rf.ctime(detector, day)
echans = ctime_data[0]
total_counts = ctime_data[1]
echan_counts = ctime_data[2]
total_rate = ctime_data[3]
echan_rate = ctime_data[4]
bin_time = ctime_data[5]
good_time = ctime_data[6]
exptime = ctime_data[7]
bin_time_mid = np.array((bin_time[:,0]+bin_time[:,1])/2)

counts = echan_rate[echan]

#read the satellite data
sat_data = rf.poshist_bin(day, bin_time_mid, detector, data_type)
sat_time_bin = sat_data[0]
sat_pos_bin = sat_data[1]
sat_lat_bin = sat_data[2]
sat_lon_bin = sat_data[3]
sat_q_bin = sat_data[4]

#calculate the earth data
earth_data = calc.earth_ang_bin(detector, day, bin_time_mid, data_type)
earth_ang_bin = earth_data[0]
earth_ang_eff = calc.ang_eff(earth_ang_bin, echan)[0]
earth_occ_bin = calc.earth_occ_eff(earth_ang_bin, echan)

earth_ang_bin[np.where(total_rate == 0)] = 0
earth_ang_eff[np.where(total_rate == 0)] = 0
earth_occ_bin[np.where(total_rate == 0)] = 0


#### plotting ####
plot_time_bin_date = calc.met_to_date(bin_time_mid)[0]
plot_time_bin = (plot_time_bin_date - calc.day_to_met(day)[1])*24#Time of day in hours
plot_time_sat_date = calc.met_to_date(sat_time_bin)[0]
plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24#Time of day in hours


###plot each on the same axis as converted to counts###
fig, ax1 = plt.subplots()

#add two independent y-axes
ax2 = ax1.twinx()
ax3 = ax1.twinx()
axes = [ax1, ax2, ax3]

#Make some space on the right side for the extra y-axis
fig.subplots_adjust(right=0.75)

# Move the last y-axis spine over to the right by 20% of the width of the axes
#axes[-1].spines['right'].set_position(('axes', 1.2))

# To make the border of the right-most axis visible, we need to turn the frame on. This hides the other plots, however, so we need to turn its fill off.
#axes[-1].set_frame_on(True)
#axes[-1].patch.set_visible(False)

plot1 = ax1.step(plot_time_bin, counts, 'b-', label = 'Count rate', alpha=0.2)
plot3 = ax3.plot(plot_time_sat, 12+0.9*earth_ang_bin, 'r-', label = 'Earth-detector angle', linewidth=2)
plot4 = ax3.plot(plot_time_sat, 150-1.5*earth_ang_eff, 'g-', label = 'effective Earth-detector angle', linewidth=2)
plot2 = ax2.plot(plot_time_sat, 0.8-1.1*earth_occ_bin, 'y-', label = 'Earth occultation', linewidth=2)

plots = plot1 + plot2 + plot3 + plot4
labels = [l.get_label() for l in plots]
ax3.legend(plots, labels, loc=1, fontsize=30)

ax1.grid()

ax1.set_xlabel('Time of day in 24h', fontsize=40)
ax1.set_ylabel('Count rate', fontsize=40)
#ax2.set_ylabel('Effective area fraction', fontsize=20)
#ax3.set_ylabel('Angle in degrees', fontsize=20)

ax2.axes.get_yaxis().set_visible(False)
ax3.axes.get_yaxis().set_visible(False)

ax1.set_xlim([2.4, 10.3])
ax1.set_ylim([40, 410])
#ax2.set_xlim([0, 24.1])
ax2.set_ylim([0, 1])
#ax3.set_xlim([0, 24.1])
ax3.set_ylim([-20, 230])
#ax4.set_xlim([0, 24.1])
#ax4.set_ylim([-1.5, 1.5])

ax1.tick_params(axis='both', which='major', labelsize=30)
ax2.tick_params(axis='both', which='major', labelsize=15)
ax3.tick_params(axis='both', which='major', labelsize=15)

mpl.pyplot.rcParams['agg.path.chunksize'] = 20000

plt.title('Earth Occultation and angles for the ' + detector + '-detector on the ' + ordinal(int(str(day)[4:6])) + ' ' + date.strftime('%B')[0:3] + ' ' + str(year), fontsize=40)

plt.show()

In [19]:
### CTIME-data & SAA plot ###

day = 100804
detector = 'nb'
echan = 3

data = rf.ctime(detector, day)
chan_energies = data[0]
total_counts = data[1]
echan_counts = data[2]
total_rate = data[3]
echan_rate = data[4]
bin_time = data[5]
good_time = data[6]
exptime = data[7]
bin_time_mid = np.array((bin_time[:,0]+bin_time[:,1])/2)

counts = echan_rate[echan]

year = int('20' + str(day)[0:2])
#get the iso-date-format from the day
date = datetime(year, int(str(day)[2:4]), int(str(day)[4:6]))
#get the ordinal indicator for the date
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])

fit_curve = calc.curve_fit_plots(day, detector, echan)[2]



#### plotting ####
plot_time_bin_date = calc.met_to_date(bin_time_mid)[0]
plot_time_bin = (plot_time_bin_date - calc.day_to_met(day)[1])*24#Time of day in hours


#plt.plot(plot_time_bin, counts, 'b-', label = 'Countrate')
plt.step(plot_time_bin, total_counts, 'b', label = 'Count rate')
plt.plot(plot_time_bin, fit_curve, 'r', label = 'Fitted Background Curve')

plt.legend(fontsize=30)
plt.grid()

#plt.xlim([21.5, 24])
plt.xlim([-0.5, 24.5])
plt.ylim([0, 1100])

plt.xlabel('Time of day in 24h', fontsize=40)
plt.ylabel('Countrate', fontsize=40)
plt.tick_params(axis='both', which='major', labelsize=30)

plt.title(data_type + '-count-rate for the ' + detector + '-detector on the ' + ordinal(int(str(day)[4:6])) + ' ' + date.strftime('%B')[0:3] + ' ' + str(year), fontsize=40)

plt.show()

In [ ]:
### McIlwain L-parameter plot ###

echan = 6

#read the measurement data
ctime_data = rf.ctime(detector, day)
chan_energies = ctime_data[0]
total_counts = ctime_data[1]
echan_counts = ctime_data[2]
total_rate = ctime_data[3]
echan_rate = ctime_data[4]
bin_time = ctime_data[5]
good_time = ctime_data[6]
exptime = ctime_data[7]
bin_time_mid = np.array((bin_time[:,0]+bin_time[:,1])/2)

counts = echan_rate[echan]

year = int('20' + str(day)[0:2])
#get the iso-date-format from the day
date = datetime(year, int(str(day)[2:4]), int(str(day)[4:6]))
#get the ordinal indicator for the date
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])

#read the satellite data
sat_data = rf.poshist_bin(day, bin_time_mid, detector, data_type)
sat_time_bin = sat_data[0]
sat_pos_bin = sat_data[1]
sat_lat_bin = sat_data[2]
sat_lon_bin = sat_data[3]
sat_q_bin = sat_data[4]

#read the mcilwain parameter data
sat_time = readfile.poshist(readfile(), day)[0]
lat_data = readfile.mcilwain(readfile(), day)
mc_b = lat_data[1]
mc_l = lat_data[2]
mc_b = calculate.intpol(calculate(), mc_b, day, 0, sat_time, bin_time_mid)[0]
mc_l = calculate.intpol(calculate(), mc_l, day, 0, sat_time, bin_time_mid)[0]

magnetic = mc_l
magnetic = magnetic - np.mean(magnetic, dtype=np.float64)
magnetic[np.where(total_rate == 0)] = 0


#### plotting ####
plot_time_bin_date = calc.met_to_date(bin_time_mid)[0]
plot_time_bin = (plot_time_bin_date - calc.day_to_met(day)[1])*24#Time of day in hours
plot_time_sat_date = calc.met_to_date(sat_time_bin)[0]
plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24#Time of day in hours


###plot each on the same axis as converted to counts###
fig, ax1 = plt.subplots()

#add two independent y-axes
ax2 = ax1.twinx()
axes = [ax1, ax2]

#Make some space on the right side for the extra y-axis
#fig.subplots_adjust(right=0.75)

# Move the last y-axis spine over to the right by 20% of the width of the axes
#axes[-1].spines['right'].set_position(('axes', 1.2))

# To make the border of the right-most axis visible, we need to turn the frame on. This hides the other plots, however, so we need to turn its fill off.
#axes[-1].set_frame_on(True)
#axes[-1].patch.set_visible(False)

plot1 = ax1.step(plot_time_bin, counts, 'b-', label = 'Count rate', alpha=0.2)
plot2 = ax2.plot(plot_time_sat, 0.9*magnetic, 'g-', label = 'McIlwain L-parameter', linewidth=2)

plots = plot1 + plot2
labels = [l.get_label() for l in plots]
ax2.legend(plots, labels, loc=1, fontsize=30)

ax1.grid()

ax1.set_xlabel('Time of day in 24h', fontsize=40)
ax1.set_ylabel('Count rate', fontsize=40)
#ax2.set_ylabel('Effective area fraction', fontsize=20)

ax2.axes.get_yaxis().set_visible(False)

ax1.set_xlim([15, 20])
#ax1.set_xlim([-0.5, 24.5])
ax1.set_ylim([0, 230])
#ax2.set_xlim([0, 24.1])
ax2.set_ylim([-0.5, 1])

ax1.tick_params(axis='both', which='major', labelsize=30)
ax2.tick_params(axis='both', which='major', labelsize=30)

plt.title('Count rate and the McIlwain L-parameter for the ' + detector + '-detector on the ' + ordinal(int(str(day)[4:6])) + ' ' + date.strftime('%B')[0:3] + ' ' + str(year), fontsize=40)

plt.show()

print len(counts)

In [63]:
### CGB from Ajello paper ###

energies1 = np.arange(10000) + 1

C1 = 0.1015

E_B1 = 29.99

R_11 = 1.32

R_21 = 2.88

spectrum1 = (energies1**2)*C1/((energies1/E_B1)**R_11 + (energies1/E_B1)**R_21)
intensity1 = C1/((energies1/E_B1)**R_11 + (energies1/E_B1)**R_21)

#plt.plot(energies1, spectrum1, label = 'CGB', linewidth = 2)
plt.plot(energies1, intensity1, 'b', label = 'CGB', linewidth = 2)

R_11_2 = 0.9*1.32

R_21_2 = 0.8*2.88

spectrum1_2 = (energies1**2)*C1/((energies1/E_B1)**R_11_2 + (energies1/E_B1)**R_21_2)
intensity1_2 = C1/((energies1/E_B1)**R_11 + (energies1/E_B1)**R_21_2)

#plt.plot(energies1, spectrum1, label = 'CGB', linewidth = 2)
plt.plot(energies1, intensity1_2, 'r', label = 'CGB adjusted', linewidth = 2)



#energies2 = np.arange(10000) + 1

#C2 = 0.109

#E_B2 = 28.

#R_12 = 1.40

#R_22 = 2.88

#spectrum2 = (energies2**2)*C2/((energies2/E_B2)**R_12 + (energies2/E_B2)**R_22)

#plt.plot(energies2, spectrum2, label = 'CGB Tuerler 2010', linewidth = 2)



energies3 = np.arange(10000) + 1

C3 = 0.0148

E_B3 = 33.7

R_13 = -5.

R_23 = 1.72

spectrum3 = (energies3**2)*C3/((energies3/E_B3)**R_13 + (energies3/E_B3)**R_23)
intensity3 = C3/((energies3/E_B3)**R_13 + (energies3/E_B3)**R_23)

#plt.plot(energies3, spectrum3, label = 'Earth albedo', linewidth = 2)
#plt.plot(energies3, intensity3, label = 'Earth albedo', linewidth = 2)



earth_effect = np.fabs(intensity3 - intensity1)

zero = energies3[np.where(earth_effect < 0.0001)]
#print zero[:15]

#plt.plot(energies1, earth_effect, label = 'Earth occultation effect', linewidth = 2)


plt.xscale('log')
plt.yscale('log')
plt.legend(fontsize=30)
plt.grid()

plt.xlim(10, 1000)
plt.ylim(0.0001, 1)

plt.xlabel('Energy in keV', fontsize=40)
plt.ylabel('Intensity in [ph cm$^{-2}$ s$^{-1}$ keV$^{-1}$ sr$^{-1}$]', fontsize=40)
plt.tick_params(axis='both', which='major', labelsize=30)

#plt.title('CGB spectrum vs. the Earth albedo spectrum', fontsize=40)
plt.title('CGB spectrum vs. coefficient spectrum', fontsize=40)

plt.show()

In [ ]:
### CTIME-Channels gewichteter Mittelwert ###

#Read the data file
user = getpass.getuser()
path = '/home/' + user + '/Documents/Plot_digitalizer/'
filepath = os.path.join(path, 'NaI_total.dat')
f1 = open(filepath)
#store the file contents in a list of every row
lines = f1.readlines()[1:]
f1.close()

#just some variables going to be arrays of our data
x1 = []
y1 = []

#scan each line in the list, split the string into two and add them to the arrays
for line in lines:
    p = line.split()
    x1.append(float(p[0]))
    y1.append(float(p[1]))

xv = np.array(x1)
yv = np.array(y1)

energies = np.arange(1997) + 4

tck = interpolate.splrep(xv, yv, s=1.12)
area = interpolate.splev(energies, tck, der=0)

area[:4] = np.array([0.058, 0.237, 0.802, 2.212])

#plt.plot(xv, yv, 'r--')
#plt.plot(energies, area, 'b-')

#labeling the axes
#plt.ylabel('Effective Area in cm^2')
#plt.xlabel('Energy in keV')

#plt.ylim(0.01, 30)

#plt.xscale('log')
#plt.yscale('log')

#anzeigen lassen
#plt.show()

energy_boundaries = np.array([985,2000])

chan_en = energies[np.where(np.logical_and(energies >= energy_boundaries[0], energies <= energy_boundaries[1]))]

chan_ar = area[np.where(np.logical_and(energies >= energy_boundaries[0], energies <= energy_boundaries[1]))]

chan_mid = np.sum((chan_ar)*(chan_en))/((np.sum(chan_ar)))

print chan_mid

In [ ]:
### CTIME angular dependency from DRM ###

fitsname = 'peak_eff_area_angle_calib_GBM_all_DRM.fits'
user = getpass.getuser()
fits_path = '/home/' + user + '/Work/calibration/'
fitsfilepath = os.path.join(fits_path, fitsname)
fitsfile = fits.open(fitsfilepath, mode='update')
data = fitsfile[1].data
fitsfile.close()
x0 = data.field(0)
y0 = data.field(1)#for NaI (4-12 keV) gewichteter Mittelwert = 10.76 keV
y1 = data.field(2)#for NaI (12-27 keV) gewichteter Mittelwert = 20.42 keV
y2 = data.field(3)#for NaI (27-50 keV) gewichteter Mittelwert = 38.80 keV
y3 = data.field(4)#for NaI (50-102 keV) gewichteter Mittelwert = 76.37 keV
y4 = data.field(5)#for NaI (102-295 keV) gewichteter Mittelwert = 190.19 keV
y5 = data.field(6)#for NaI (295-540 keV) gewichteter Mittelwert = 410.91 keV
y6 = data.field(7)#for NaI (540-985 keV) gewichteter Mittelwert = 751.94 keV
y7 = data.field(8)#for NaI (985-2000 keV) gewichteter Mittelwert = 1466.43 keV

x = ((np.arange(3601) - 1800.)/10.)

tck = interpolate.splrep(x0, y0)
y0 = interpolate.splev(x, tck, der=0)
tck = interpolate.splrep(x0, y1)
y1 = interpolate.splev(x, tck, der=0)
tck = interpolate.splrep(x0, y2)
y2 = interpolate.splev(x, tck, der=0)
tck = interpolate.splrep(x0, y3)
y3 = interpolate.splev(x, tck, der=0)
tck = interpolate.splrep(x0, y4)
y4 = interpolate.splev(x, tck, der=0)
tck = interpolate.splrep(x0, y5)
y5 = interpolate.splev(x, tck, der=0)
tck = interpolate.splrep(x0, y6)
y6 = interpolate.splev(x, tck, der=0)
tck = interpolate.splrep(x0, y7)
y7 = interpolate.splev(x, tck, der=0)



mpl.rcParams['axes.color_cycle'] = ['b', 'g', 'r', 'c', 'm', 'y', 'k', '0.5']

plt.plot(x, y0, label = '0')
plt.plot(x, y1, label = '1')
plt.plot(x, y2, label = '2')
plt.plot(x, y3, label = '3')
plt.plot(x, y4, label = '4')
plt.plot(x, y5, label = '5')
plt.plot(x, y6, label = '6')
plt.plot(x, y7, label = '7')



plt.tick_params(axis='both', which='major', labelsize=30)

plt.xlabel('Angle in degrees', fontsize=40)
plt.ylabel('Effective area in cm$^2$', fontsize=40)
plt.grid()
plt.legend(fontsize=30)
plt.show()

In [ ]:
### DDBF Plot ###

day = 150926

sources_data = rf.point_sources()
sources_names = sources_data[0]
sources_coordinates = sources_data[1]
sources_number = int(len(sources_names))

year = int('20' + str(day)[0:2])

#get the iso-date-format from the day
date = datetime(year, int(str(day)[2:4]), int(str(day)[4:6]))

#get the ordinal indicator for the date
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])

detector_name = 'n5'
echan = 2
data_type = 'ctime'

#read the measurement data
ctime_data = rf.ctime(detector_name, day)
chan_energies = ctime_data[0]
total_counts = ctime_data[1]
echan_counts = ctime_data[2]
total_rate = ctime_data[3]
echan_rate = ctime_data[4]
bin_time = ctime_data[5]
good_time = ctime_data[6]
exptime = ctime_data[7]
bin_time_mid = np.array((bin_time[:,0]+bin_time[:,1])/2)

total_rate = np.sum(echan_rate[1:-1], axis = 0)

counts = echan_rate[echan]

#read the satellite data
sat_data = rf.poshist_bin(day, bin_time_mid, detector_name, data_type)
sat_time_bin = sat_data[0]
sat_pos_bin = sat_data[1]
sat_lat_bin = sat_data[2]
sat_lon_bin = sat_data[3]
sat_q_bin = sat_data[4]

#calculate the sun data
sun_data = calc.sun_ang_bin(detector_name, day, bin_time_mid, data_type)
sun_ang_bin = sun_data[0]
sun_ang_bin = calc.ang_eff(sun_ang_bin, echan)[0]
sun_rad = sun_data[2]
sun_ra = sun_rad[:,0]
sun_dec = sun_rad[:,1]
sun_occ = calc.src_occultation_bin(day, sun_ra, sun_dec, bin_time_mid)[0]

#sun_max = calc.ang_eff(0., echan, data_type)[0]
sun_max = 126.6768698 

#calculate the earth data
earth_data = calc.earth_ang_bin(detector_name, day, bin_time_mid, data_type)
earth_ang_bin = earth_data[0]
#earth_ang_bin = calc.ang_eff(earth_ang_bin, echan)[0]
earth_ang_bin = calc.earth_occ_eff(earth_ang_bin, echan)

earth_max = calc.earth_occ_eff(0., echan)

#calculate the pointsources data
sources_ang_bin = np.array([np.zeros(len(sun_ang_bin))])

for i in range(0, sources_number):
    src_ra = sources_coordinates[0,i]
    src_dec = sources_coordinates[1,i]
    
    src_data = calc.burst_ang_bin(detector_name, day, src_ra, src_dec, bin_time_mid, data_type)
    src_ang_bin = src_data[0]
    src_ang_bin = calc.ang_eff(src_ang_bin, echan, data_type)[0]
    src_occ = calc.src_occultation_bin(day, src_ra, src_dec, bin_time_mid, detector_name, data_type)[0]
    
    src_ang_bin[np.where(src_occ == 0)] = 0
    src_ang_bin[np.where(total_rate == 0)] = 0
    
    #remove vertical movement from scaling the src_ang_bin functions
    #src_ang_bin[src_ang_bin>0] = src_ang_bin[src_ang_bin>0] - np.min(src_ang_bin[src_ang_bin>0])
    
    src_ang_bin = np.array(src_ang_bin)
    
    sources_ang_bin = np.concatenate((sources_ang_bin, [src_ang_bin]), axis = 0)

sources_ang_bin = np.delete(sources_ang_bin, 0, 0)

sources_frac_bin = sources_ang_bin/sun_max

#read the SFL data
flares = rf.flares(year)
flares_day = flares[0]
flares_time = flares[1]
if np.any(flares_day == day) == True:
    flares_today = flares_time[:,np.where(flares_day == day)]
    flares_today = np.squeeze(flares_today, axis=(1,))/3600.
else:
    flares_today = np.array(-5)

#read the mcilwain parameter data
sat_time = readfile.poshist(readfile(), day)[0]
lat_data = readfile.mcilwain(readfile(), day)
mc_b = lat_data[1]
mc_l = lat_data[2]

mc_b = calc.intpol(mc_b, day, 0, sat_time, bin_time_mid)[0]
mc_l = calc.intpol(mc_l, day, 0, sat_time, bin_time_mid)[0]

magnetic = mc_l
#magnetic = magnetic - np.mean(magnetic, dtype=np.float64)

magnetic_max = 3.

#constant function corresponding to the diffuse y-ray background
cgb = np.ones(len(total_rate))

cgb_max = 2.

#counts[120000:] = 0
cgb[np.where(total_rate == 0)] = 0
earth_ang_bin[np.where(total_rate == 0)] = 0
sun_ang_bin[np.where(sun_occ == 0)] = 0
sun_ang_bin[np.where(total_rate == 0)] = 0
magnetic[np.where(total_rate == 0)] = 0

sources_curve = np.squeeze(np.sum(sources_ang_bin, axis = 0))
sources_frac_curve = np.squeeze(np.sum(sources_frac_bin, axis = 0))/sources_number


#convert the x-axis into hours of the day
plot_time_bin_date = calc.met_to_date(bin_time_mid)[0]
plot_time_bin = (plot_time_bin_date - calc.day_to_met(day)[1])*24#Time of day in hours
plot_time_sat_date = calc.met_to_date(sat_time_bin)[0]
plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24#Time of day in hours


fig, ax1 = plt.subplots()

#plot1 = ax1.plot(plot_time_bin, counts, 'b-', label = 'Countrate')
#plot2 = ax1.plot(plot_time_sat, sun_ang_bin, 'y-', label = 'Sun effective angle')
#plot3 = ax1.plot(plot_time_sat, earth_ang_bin, 'c-', label = 'Earth occultation')
#plot4 = ax1.plot(plot_time_sat, magnetic, 'g-', label = 'McIlwain L-parameter')
#plot5 = ax1.plot(plot_time_sat, cgb, 'b--', label = 'CGB')
#plot6 = ax1.plot(plot_time_sat, sources_curve, 'k-', label = 'Point-like sources')

#plot1 = ax1.plot(plot_time_bin, counts, 'b-', label = 'Count rate')
plot2 = ax1.plot(plot_time_sat, sun_ang_bin/sun_max, 'y-', label = 'Sun effective angle')
plot3 = ax1.plot(plot_time_sat, earth_ang_bin/earth_max, 'c-', label = 'Earth occultation')
plot4 = ax1.plot(plot_time_sat, magnetic/magnetic_max, 'g-', label = 'McIlwain L-parameter')
plot5 = ax1.plot(plot_time_sat, cgb/cgb_max, 'b--', label = 'CGB')
plot6 = ax1.plot(plot_time_sat, sources_frac_curve, 'k-', label = 'Point-like sources')

#plot vertical lines for the solar flares of the day
if np.all(flares_today != -5):
    if len(flares_today[0]) > 1:
        for i in range(0, len(flares_today[0])):
            plt.axvline(x = flares_today[0,i], ymin = 0., ymax = 1., linewidth=2, color = 'grey')
            plt.axvline(x = flares_today[1,i], ymin = 0., ymax = 1., color = 'grey', linestyle = '--')
    else:
        plt.axvline(x = flares_today[0], ymin = 0., ymax = 1., linewidth=2, color = 'grey')
        plt.axvline(x = flares_today[1], ymin = 0., ymax = 1., color = 'grey', linestyle = '--')

plots = plot2 + plot3 + plot4 + plot5 + plot6
labels = [l.get_label() for l in plots]
ax1.legend(plots, labels, loc=1, fontsize=30)

ax1.grid()

ax1.set_xlabel('Time of day in 24h', fontsize=40)
ax1.set_ylabel('Count rate', fontsize=40)
ax1.tick_params(axis='both', which='major', labelsize=30)

#ax1.set_xlim([9.84, 9.85])
ax1.set_xlim([-0.5, 24.5])
ax1.set_ylim([-0.1, 1])

plt.title('Background components for the ' + detector_name + '-detector on the ' + ordinal(int(str(day)[4:6])) + ' ' + date.strftime('%B')[0:3] + ' ' + str(year), fontsize=40)

plt.show()

In [148]:
### IceCube Neutrino Event Winkel ### eigentlich Sonnen-Event

detectors = np.array(['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'na', 'nb'])
#detectors = np.array(['n0', 'n1', 'n3', 'n4', 'n6', 'n7', 'n8', 'n9', 'nb'])
#detectors = np.array(['n0', 'n1', 'n3', 'n4', 'n6', 'n7', 'n8', 'n9', 'na', 'nb'])

day = 111209
burst_ra = 14.3462
burst_dec = -46.8005

start_time = 6.6
end_time = 8.3

#v_line = 4.90648687098
from itertools import cycle

lines = ["-","-","-","-","-","-","-","--","--","--","--","--","--","--"]
linecycler = cycle(lines)

burst_occ = calc.src_occultation(day, burst_ra, burst_dec)[0]

sun = calc.sun_pos(day)[0]
burst_pos = np.array([np.cos(burst_ra*(2.*math.pi)/360.)*np.cos(burst_dec*(2.*math.pi)/360.), np.sin(burst_ra*(2.*math.pi)/360.)*np.cos(burst_dec*(2.*math.pi)/360.), np.sin(burst_dec*(2.*math.pi)/360.)], float) #unit-vector pointing to the burst location
scalar_product = sun[0]*burst_pos[0] + sun[1]*burst_pos[1] + sun[2]*burst_pos[2]
sun_ang = np.arccos(scalar_product)
sun_ang = (sun_ang)*360./(2.*math.pi) #convert to degrees
sun_ang = np.array(sun_ang)

sat_time = rf.poshist(day)[0]

plot_time_sat_date = calc.met_to_date(sat_time)[0]
plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24#Time of day in hours

#plot_seconds = (plot_time_sat - v_line)*3600.

fig = plt.figure()
ax = fig.add_subplot(111)

mpl.rcParams['axes.color_cycle'] = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

for i in range(0, len(detectors)):
    #print detectors[i]
    det_data = calc.burst_ang(detectors[i], day, burst_ra, burst_dec)
    det_ang = det_data[0]
    #sat_time = det_data[1]
    #det_ang[np.where(det_ang > 90.)] = 0.
    
    plt.plot(plot_time_sat, det_ang, next(linecycler), label = detectors[i])

plt.axvline(x = 6.6744, ymin = 0., ymax = 1., linewidth=2, color = 'grey')
plt.axvline(x = 7.20222, ymin = 0., ymax = 1., linewidth=2, color = 'grey')
plt.axvline(x = 7.6050, ymin = 0., ymax = 1., linewidth=2, color = 'grey')
plt.axvline(x = 7.7578, ymin = 0., ymax = 1., linewidth=2, color = 'grey')
plt.axvline(x = 7.9939, ymin = 0., ymax = 1., linewidth=2, color = 'grey')
plt.axvline(x = 8.2300, ymin = 0., ymax = 1., linewidth=2, color = 'grey')

plt.plot(plot_time_sat, burst_occ, next(linecycler), label = 'Earth')
#plt.plot(plot_time_sat, sun_ang, 'b-.',label = 'Sun')
leg = plt.legend(fontsize=20, loc=2)
plt.grid()
plt.tick_params(axis='both', which='major', labelsize=30)

plt.xlabel('Time in 24 h', fontsize=40)
plt.ylabel('Angle in degrees', fontsize=40)
plt.title('Angles between the ULGRB 111209 and the NaI-detectors', fontsize=45)

#plt.xlim([-1000, 1000])
plt.xlim([start_time, end_time])
plt.ylim([-10, 200])


#move legend
plt.draw() # Draw the figure so you can find the positon of the legend. 

# Get the bounding box of the original legend
bb = leg.legendPatch.get_bbox().inverse_transformed(ax.transAxes)

# Change to location of the legend. 
xOffset = .1
newX0 = bb.x0 + xOffset
newX1 = bb.x1 + xOffset
bb.set_points([[newX0, bb.y0], [newX1, bb.y1]])
leg.set_bbox_to_anchor(bb)



plt.show()

In [ ]:
### GW Event Winkel ###

detectors = np.array(['n2', 'n4', 'n5', 'n8', 'na', 'nb'])
#detectors = np.array(['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'na', 'nb'])

day = 150914
burst_ra = 75
burst_dec = -73

start_time = 9.75
end_time = 9.95

v_line = 9.84605472222


burst_occ = calc.src_occultation(day, burst_ra, burst_dec)[0]

sat_time = rf.poshist(day)[0]

plot_time_sat_date = calc.met_to_date(sat_time)[0]
plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24#Time of day in hours

mpl.rcParams['axes.color_cycle'] = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

for i in range(0, len(detectors)):
    #print detectors[i]
    det_data = calc.burst_ang(detectors[i], day, burst_ra, burst_dec)
    det_ang = det_data[0]
    #sat_time = det_data[1]
    #det_ang[np.where(det_ang > 90.)] = 0.
    
    plt.plot(plot_time_sat, det_ang, label = detectors[i])
    
plt.axvline(x = v_line, ymin = 0., ymax = 1., linewidth=2, color = 'grey')

plt.plot(plot_time_sat, burst_occ, label = 'Earth')    
plt.legend(fontsize=30)
plt.grid()
plt.tick_params(axis='both', which='major', labelsize=30)

plt.xlabel('Daytime in 24h', fontsize=40)
plt.ylabel('Angle in degrees', fontsize=40)
plt.title('Winkelverhaeltnisse zwischen dem IceCube Neutrino-Event und den Fermi-Detektoren', fontsize=40)

plt.xlim([start_time, end_time])
plt.ylim([-10, 180])

plt.show()

In [ ]:
### GW-DDBF Plot ###

day = 150914

sources_data = rf.point_sources()
sources_names = sources_data[0]
sources_coordinates = sources_data[1]
sources_number = int(len(sources_names))

year = int('20' + str(day)[0:2])

#get the iso-date-format from the day
date = datetime(year, int(str(day)[2:4]), int(str(day)[4:6]))

#get the ordinal indicator for the date
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])

detector_name = 'n5'
echan = 5
data_type = 'ctime'

#read the measurement data
ctime_data = rf.ctime(detector_name, day)
chan_energies = ctime_data[0]
total_counts = ctime_data[1]
echan_counts = ctime_data[2]
total_rate = ctime_data[3]
echan_rate = ctime_data[4]
bin_time = ctime_data[5]
good_time = ctime_data[6]
exptime = ctime_data[7]
bin_time_mid = np.array((bin_time[:,0]+bin_time[:,1])/2)

total_rate = np.sum(echan_rate[1:-1], axis = 0)

counts = echan_rate[echan]

#read the satellite data
sat_data = rf.poshist_bin(day, bin_time_mid, detector_name, data_type)
sat_time_bin = sat_data[0]
sat_pos_bin = sat_data[1]
sat_lat_bin = sat_data[2]
sat_lon_bin = sat_data[3]
sat_q_bin = sat_data[4]

#calculate the sun data
sun_data = calc.sun_ang_bin(detector_name, day, bin_time_mid, data_type)
sun_ang_bin = sun_data[0]
sun_ang_bin = calc.ang_eff(sun_ang_bin, echan)[0]
sun_rad = sun_data[2]
sun_ra = sun_rad[:,0]
sun_dec = sun_rad[:,1]
sun_occ = calc.src_occultation_bin(day, sun_ra, sun_dec, bin_time_mid)[0]

#sun_max = calc.ang_eff(0., echan, data_type)[0]
sun_max = 126.6768698 

#calculate the earth data
earth_data = calc.earth_ang_bin(detector_name, day, bin_time_mid, data_type)
earth_ang_bin = earth_data[0]
#earth_ang_bin = calc.ang_eff(earth_ang_bin, echan)[0]
earth_ang_bin = calc.earth_occ_eff(earth_ang_bin, echan)

earth_max = calc.earth_occ_eff(0., echan)

#calculate the pointsources data
sources_ang_bin = np.array([np.zeros(len(sun_ang_bin))])

for i in range(0, sources_number):
    src_ra = sources_coordinates[0,i]
    src_dec = sources_coordinates[1,i]
    
    src_data = calc.burst_ang_bin(detector_name, day, src_ra, src_dec, bin_time_mid, data_type)
    src_ang_bin = src_data[0]
    src_ang_bin = calc.ang_eff(src_ang_bin, echan, data_type)[0]
    src_occ = calc.src_occultation_bin(day, src_ra, src_dec, bin_time_mid, detector_name, data_type)[0]
    
    src_ang_bin[np.where(src_occ == 0)] = 0
    src_ang_bin[np.where(total_rate == 0)] = 0
    
    #remove vertical movement from scaling the src_ang_bin functions
    #src_ang_bin[src_ang_bin>0] = src_ang_bin[src_ang_bin>0] - np.min(src_ang_bin[src_ang_bin>0])
    
    src_ang_bin = np.array(src_ang_bin)
    
    sources_ang_bin = np.concatenate((sources_ang_bin, [src_ang_bin]), axis = 0)

sources_ang_bin = np.delete(sources_ang_bin, 0, 0)

sources_frac_bin = sources_ang_bin/sun_max

#read the SFL data
flares = rf.flares(year)
flares_day = flares[0]
flares_time = flares[1]
if np.any(flares_day == day) == True:
    flares_today = flares_time[:,np.where(flares_day == day)]
    flares_today = np.squeeze(flares_today, axis=(1,))/3600.
else:
    flares_today = np.array(-5)

#read the mcilwain parameter data
sat_time = readfile.poshist(readfile(), day)[0]
lat_data = readfile.mcilwain(readfile(), day)
mc_b = lat_data[1]
mc_l = lat_data[2]

mc_b = calc.intpol(mc_b, day, 0, sat_time, bin_time_mid)[0]
mc_l = calc.intpol(mc_l, day, 0, sat_time, bin_time_mid)[0]

magnetic = mc_l
#magnetic = magnetic - np.mean(magnetic, dtype=np.float64)

magnetic_max = 3.

#constant function corresponding to the diffuse y-ray background
cgb = np.ones(len(total_rate))

cgb_max = 2.

#counts[120000:] = 0
cgb[np.where(total_rate == 0)] = 0
earth_ang_bin[np.where(total_rate == 0)] = 0
sun_ang_bin[np.where(sun_occ == 0)] = 0
sun_ang_bin[np.where(total_rate == 0)] = 0
magnetic[np.where(total_rate == 0)] = 0

sources_curve = np.squeeze(np.sum(sources_ang_bin, axis = 0))
sources_frac_curve = np.squeeze(np.sum(sources_frac_bin, axis = 0))/sources_number


#convert the x-axis into hours of the day
plot_time_bin_date = calc.met_to_date(bin_time_mid)[0]
plot_time_bin = (plot_time_bin_date - calc.day_to_met(day)[1])*24#Time of day in hours
plot_time_sat_date = calc.met_to_date(sat_time_bin)[0]
plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24#Time of day in hours

gw_time = 9.845944

plot_seconds = (plot_time_sat - gw_time)*3600.

fig, ax1 = plt.subplots()

#plot1 = ax1.plot(plot_time_bin, counts, 'b-', label = 'Countrate')
#plot2 = ax1.plot(plot_time_sat, sun_ang_bin, 'y-', label = 'Sun effective angle')
#plot3 = ax1.plot(plot_time_sat, earth_ang_bin, 'c-', label = 'Earth occultation')
#plot4 = ax1.plot(plot_time_sat, magnetic, 'g-', label = 'McIlwain L-parameter')
#plot5 = ax1.plot(plot_time_sat, cgb, 'b--', label = 'CGB')
#plot6 = ax1.plot(plot_time_sat, sources_curve, 'k-', label = 'Point-like sources')

##plot1 = ax1.plot(plot_time_bin, counts, 'b-', label = 'Count rate')
#plot2 = ax1.plot(plot_time_sat, sun_ang_bin/sun_max, 'y-', label = 'Sun effective angle')
#plot3 = ax1.plot(plot_time_sat, earth_ang_bin/earth_max, 'c-', label = 'Earth occultation')
#plot4 = ax1.plot(plot_time_sat, magnetic/magnetic_max, 'g-', label = 'McIlwain L-parameter')
#plot5 = ax1.plot(plot_time_sat, cgb/cgb_max, 'b--', label = 'CGB')
#plot6 = ax1.plot(plot_time_sat, sources_frac_curve, 'k-', label = 'Point-like sources')
#plt.axvline(x = gw_time, ymin = 0., ymax = 1., linewidth=2, color = 'grey')

#seconds
plot2 = ax1.plot(plot_seconds, sun_ang_bin/sun_max, 'y-', label = 'Sun effective angle')
plot3 = ax1.plot(plot_seconds, earth_ang_bin/earth_max, 'c-', label = 'Earth occultation')
plot4 = ax1.plot(plot_seconds, magnetic/magnetic_max, 'g-', label = 'McIlwain L-parameter')
plot5 = ax1.plot(plot_seconds, cgb/cgb_max, 'b--', label = 'CGB')
plot6 = ax1.plot(plot_seconds, sources_frac_curve, 'k-', label = 'Point-like sources')
plt.axvline(x = 0, ymin = 0., ymax = 1., linewidth=2, color = 'grey')

#plot vertical lines for the solar flares of the day
#if np.all(flares_today != -5):
#    if len(flares_today[0]) > 1:
#        for i in range(0, len(flares_today[0])):
#            plt.axvline(x = flares_today[0,i], ymin = 0., ymax = 1., linewidth=2, color = 'grey')
#            plt.axvline(x = flares_today[1,i], ymin = 0., ymax = 1., color = 'grey', linestyle = '--')
#    else:
#        plt.axvline(x = flares_today[0], ymin = 0., ymax = 1., linewidth=2, color = 'grey')
#        plt.axvline(x = flares_today[1], ymin = 0., ymax = 1., color = 'grey', linestyle = '--')

#seconds
if np.all(flares_today != -5):
    if len(flares_today[0]) > 1:
        for i in range(0, len(flares_today[0])):
            plt.axvline(x = flares_today[0,i]*3600., ymin = 0., ymax = 1., linewidth=2, color = 'grey')
            plt.axvline(x = flares_today[1,i]*3600., ymin = 0., ymax = 1., color = 'grey', linestyle = '--')
    else:
        plt.axvline(x = flares_today[0]*3600., ymin = 0., ymax = 1., linewidth=2, color = 'grey')
        plt.axvline(x = flares_today[1]*3600., ymin = 0., ymax = 1., color = 'grey', linestyle = '--')

plots = plot2 + plot3 + plot4 + plot5 + plot6
labels = [l.get_label() for l in plots]
ax1.legend(plots, labels, loc=1, fontsize=30)

ax1.grid()

ax1.set_xlabel('Time of day in 24h', fontsize=40)
ax1.set_ylabel('Count rate', fontsize=40)
ax1.tick_params(axis='both', which='major', labelsize=30)

#ax1.set_xlim([9.843166, 9.848722])
ax1.set_xlim([-1000., 1000.])
#ax1.set_xlim([-0.5, 24.5])
ax1.set_ylim([-0.1, 1])

plt.title('Background components for the ' + detector_name + '-detector on the ' + ordinal(int(str(day)[4:6])) + ' ' + date.strftime('%B')[0:3] + ' ' + str(year), fontsize=40)

plt.show()

In [2]:
### SAA coeff development ###

def saa_data(day, detector_name, echan, data_type = 'ctime', sources_number = 'None'):
    sources_data = rf.point_sources()
    sources_names = sources_data[0]
    #sources_coordinates = sources_data[1]
    if sources_number == 'None':
        sources_number = len(sources_names)
    
    #read the file
    if data_type == 'ctime':
        if echan < 8:
            fitsname = 'ctime_' + detector_name + '_e' + str(echan) + '_kt.fits'
        elif echan == 8:
            fitsname = 'ctime_' + detector_name + '_tot_kt.fits'
        else:
            print 'Invalid value for the energy channel of this data type (ctime). Please insert an integer between 0 and 9.'
            return
    elif data_type == 'cspec':
        if echan < 128:
            fitsname = 'cspec_' + detector_name + '_e' + str(echan) + '__kt.fits'
        elif echan == 128:
            fitsname = 'cspec_' + detector_name + '_tot_kt.fits'
        else:
            print 'Invalid value for the energy channel of this data type (cspec). Please insert an integer between 0 and 129.'
            return
    else:
        print 'Invalid data type: ' + str(data_type) + '\n Please insert an appropriate data type (ctime or cspec).'
        return
    
    
    fits_path = '/home/' + user + '/Work/Fits_data/' + str(day) + '/'
    if not os.access(fits_path, os.F_OK):
        os.mkdir(fits_path)
    fitsfilepath = os.path.join(fits_path, fitsname)
    
    plot_fits = fits.open(fitsfilepath)
    fit_data = plot_fits[1].data
    pointsources_data = plot_fits[2].data
    saa_data = plot_fits[3].data
    plot_fits.close()
    
    #extract the data
    residuals = fit_data.Residuals
    counts = fit_data.Count_Rate
    fit_curve = fit_data.Fitting_curve
    cgb = fit_data.CGB_curve
    magnetic = fit_data.Mcilwain_L_curve
    earth_ang_bin = fit_data.Earth_curve
    sun_ang_bin = fit_data.Sun_curve
    sources_fit_curve = fit_data.Pointsources_curve
    plot_time_bin = fit_data.Data_time
    plot_time_sat = fit_data.Parameter_time
    fit_coeffs = fit_data.FitCoefficients
    
    sources_ang_bin = []
    sources_names = sources_names.tolist()
    for i in range(0, sources_number):
        #src_array = pointsources_data[sources_names[i]]
        src_array = pointsources_data[i]
        sources_ang_bin.append(src_array)
    sources_ang_bin = np.array(sources_ang_bin)
    
    sources_coeff = pointsources_data.FitCoefficients_Pointsources
    
    saa_prefac = saa_data.Prefactor_coefficient
    saa_exp = saa_data.Exponent_coefficient
    
    return residuals, counts, fit_curve, plot_time_bin, plot_time_sat, saa_prefac, saa_exp, sources_names, sources_coeff, sources_ang_bin


days = np.array([100701, 100702, 100703, 100704, 100705, 100706, 100707, 100708, 100709, 100710, 100711, 100712, 100713, 100714, 100715, 100716, 100717, 100719, 100721, 100722, 100723, 100724, 100725, 100726, 100728, 100729, 100730, 100731, 100801, 100803, 100804, 100805, 100806, 100807, 100808, 100809, 100810])
detectors = np.array(['n8', 'nb'])
echans = np.array([3])
data_type = 'ctime'

saa_times_n8 = []
saa_times_nb = []
saa_prefacs_n8 = []
saa_prefacs_nb = []
saa_exps_n8 = []
saa_exps_nb = []
source_coeff_n8 = []
source_coeff_nb = []
#source_ang_bin_n8 = []
#source_ang_bin_nb = []

source = 'CYGX-1'

user = getpass.getuser()

for day in days:
    for detector in detectors:
        for echan in echans:
            data = saa_data(day, detector, echan, data_type)
            residuals = np.array(data[0].astype(np.float))
            counts = np.array(data[1].astype(np.float))
            fit_curve = np.array(data[2].astype(np.float))
            plot_time_bin = np.array(data[3].astype(np.float))
            plot_time_sat = np.array(data[4].astype(np.float))
            saa_prefac = np.array(data[5].astype(np.float))
            saa_exp = np.array(data[6].astype(np.float))
            sources_names = np.array(data[7])
            sources_coeff = np.array(data[8].astype(np.float))
            sources_ang_bin = np.array(data[9].astype(np.float))
            
            
            ctime_data = rf.ctime(detector, day)
            chan_energies = ctime_data[0]
            total_counts = ctime_data[1]
            echan_counts = ctime_data[2]
            total_rate = ctime_data[3]
            echan_rate = ctime_data[4]
            bin_time = ctime_data[5]
            good_time = ctime_data[6]
            exptime = ctime_data[7]
            bin_time_mid = np.array((bin_time[:,0]+bin_time[:,1])/2)
        
            total_rate = np.sum(echan_rate[1:-1], axis = 0)
            
            saa_exits = [0]
            for i in range(1, len(total_rate)):
                if np.logical_and(total_rate[i-1] == 0, total_rate[i] != 0):
                    #print i
                    saa_exits.append(int(i))
            saa_exits = np.array(saa_exits)
            
            if saa_exits[1] - saa_exits[0] < 10:
                saa_exits = np.delete(saa_exits, 0)
            
            addition_path = '/home/' + user + '/Work/Fits/SAA_additions/' + str(day) + '/'
            if counts.all == total_rate.all:
                addition_name = detector + '_add_tot.dat'
            else:
                addition_name = detector + '_add_e' + str(echan) + '.dat'
            addition_file = os.path.join(addition_path, addition_name)
            
            if os.access(addition_file, os.F_OK):
                infos = open(addition_file, 'r')
                addition = int(infos.read())
                infos.close()

            saa_exits = saa_exits + addition
            
            #saa_daytime = []
            
            #saa_daytime = np.array(saa_daytime)
            for i in range(0, len(saa_exp)):
                if detector == 'n8':
                    saa_prefacs_n8.append(saa_prefac[i])
                    saa_exps_n8.append(saa_exp[i])
                else:
                    saa_prefacs_nb.append(saa_prefac[i])
                    saa_exps_nb.append(saa_exp[i])
            for i in range(0, len(saa_exits)):
                if detector == 'n8':
                    saa_times_n8.append(bin_time_mid[saa_exits[i]])
                else:
                    saa_times_nb.append(bin_time_mid[saa_exits[i]])
            
            if detector == 'n8':
                source_coeff_n8.append(sources_coeff[np.where(sources_names == source)])
            else:
                source_coeff_nb.append(sources_coeff[np.where(sources_names == source)])
    
            

saa_times_n8 = np.array(saa_times_n8)
saa_times_nb = np.array(saa_times_nb)
saa_prefacs_n8 = np.array(saa_prefacs_n8)
saa_prefacs_nb = np.array(saa_prefacs_nb)
saa_exps_n8 = np.array(saa_exps_n8)
saa_exps_nb = np.array(saa_exps_nb)
source_coeff_n8 = np.array(source_coeff_n8)
source_coeff_nb = np.array(source_coeff_nb)

#print saa_times_n8
#print saa_times_nb
#print saa_prefacs_n8
#print saa_prefacs_nb
#print saa_exps_n8
#print saa_exps_nb


In [11]:
print len(saa_times_n8)
print len(saa_times_nb)
print saa_times_n8
print saa_times_nb

c = np.append(saa_times_n8, [0, 0, 0])
print saa_times_nb - c
#print len(saa_prefacs_n8)
#print len(saa_prefacs_nb)
#print len(saa_exps_n8)
#print len(saa_exps_nb)

for i in range(0, len(saa_times_n8)):
    plt.axvline(saa_times_n8[i], color = "grey")

for j in range(0, len(saa_times_nb)):
    plt.axvline(saa_times_nb[j], color = "black", linestyle = '--')
    
plt.show()


#plt.plot(saa_times_n8, saa_prefacs_n8, 'b-')
#plt.plot(saa_times_n8, saa_exps_n8, 'b--')
#plt.plot(saa_times_nb, saa_prefacs_nb, 'r-')
#plt.plot(saa_times_nb, saa_exps_nb, 'r--')

#plt.show()

370
373
[  2.99635201e+08   2.99639913e+08   2.99645885e+08   2.99651825e+08
   2.99657354e+08   2.99695614e+08   2.99701856e+08   2.99707921e+08
   2.99713934e+08   2.99719923e+08   2.99721600e+08   2.99725899e+08
   2.99731870e+08   2.99737774e+08   2.99781698e+08   2.99787868e+08
   2.99793916e+08   2.99799919e+08   2.99805904e+08   2.99808001e+08
   2.99811881e+08   2.99817853e+08   2.99823721e+08   2.99866556e+08
   2.99867753e+08   2.99873878e+08   2.99879912e+08   2.99885911e+08
   2.99891891e+08   2.99894400e+08   2.99897861e+08   2.99903832e+08
   2.99909575e+08   2.99947353e+08   2.99953787e+08   2.99959881e+08
   2.99965901e+08   2.99971895e+08   2.99977871e+08   2.99980800e+08
   2.99983848e+08   2.99989791e+08   2.99995362e+08   3.00033541e+08
   3.00039810e+08   3.00045879e+08   3.00051892e+08   3.00057881e+08
   3.00063857e+08   3.00067203e+08   3.00069832e+08   3.00075745e+08
   3.00081155e+08   3.00119642e+08   3.00125825e+08   3.00131878e+08
   3.00137886e+08   3.0014

In [17]:
print saa_times_nb[np.where(np.fabs(saa_times_nb - 3.02582*10**8) < 1.*10**2)]

print calc.met_to_date(saa_times_nb[np.where(np.fabs(saa_times_nb - 3.02582*10**8) < 1.*10**2)])

[  3.02582019e+08]
(array([ 55412.10667908]), <Time object: scale='tt' format='mjd' value=[ 55412.1074451]>, <Time object: scale='utc' format='isot' value=['2010-08-04T02:33:37.072']>, <Time object: scale='utc' format='iso' value=['2010-08-04 02:33:37.072']>, <Time object: scale='utc' format='decimalyear' value=[ 2010.58933337]>)


In [26]:
days_plot = np.array([100701, 100702, 100703, 100704, 100705, 100706, 100707, 100708, 100709, 100710, 100711, 100712, 100713, 100714, 100715, 100716, 100717, 100719, 100721, 100722, 100723, 100724, 100725, 100726, 100728, 100729, 100730, 100731, 100732, 100734, 100735, 100736, 100737, 100738, 100739, 100740, 100741])

plt.plot(days_plot, source_coeff_n8)
plt.plot(days_plot, source_coeff_nb)

plt.show()

In [77]:
### CGB & Earth Occultation coeff spectrum ### (viele Felder)

#Read the data file
user = getpass.getuser()
path = '/home/' + user + '/Documents/Plot_digitalizer/'
filepath = os.path.join(path, 'NaI_total.dat')
f1 = open(filepath)
#store the file contents in a list of every row
lines = f1.readlines()[1:]
f1.close()

#just some variables going to be arrays of our data
x1 = []
y1 = []

#scan each line in the list, split the string into two and add them to the arrays
for line in lines:
    p = line.split()
    x1.append(float(p[0]))
    y1.append(float(p[1]))

xv = np.array(x1)
yv = np.array(y1)

energies = np.arange(1997) + 4

tck = interpolate.splrep(xv, yv, s=1.12)
det_resp = interpolate.splev(energies, tck, der=0)

det_resp[:4] = np.array([0.058, 0.237, 0.802, 2.212])


## CGB spectrum (Ajello)
#fac = 0.9
C1 = 0.1015
E_B1 = 29.99
R_11 = 1.32
R_21 = 2.88
#R_11 = 0.9*1.32
#R_21 = 0.8*2.88

spectrum1 = (energies**2)*C1/((energies/E_B1)**R_11 + (energies/E_B1)**R_21)
intensity1 = C1/((energies/E_B1)**R_11 + (energies/E_B1)**R_21)


## Earth spectrum (Ajello)
C3 = 0.0148
E_B3 = 33.7
R_13 = -5.
R_23 = 1.72

spectrum3 = (energies**2)*C3/((energies/E_B3)**R_13 + (energies/E_B3)**R_23)
intensity3 = C3/((energies/E_B3)**R_13 + (energies/E_B3)**R_23)





earth_effect = np.fabs(intensity3 - intensity1)
#earth_effect = intensity3 - intensity1

#zero = energies[np.where(earth_effect < 0.00001)]#index 121
#print zero[:15]




In [28]:
def get_coeffs(day, detector_name, data_type = 'ctime'):
    """This function reads the fitting coefficients for the plot-fits-files and saves them in arrays of the form: cgb, magnetic, earth_ang, sun_ang, pointsources, echans\n
        Input:\n
        calculate.get_coeffs( day = YYMMDD, detector (f. e. 'n5'), data_type = 'ctime' (or 'cspec'))\n
        Output:\n
        0 = cgb\n
        1 = magnetic\n
        2 = earth_ang\n
        3 = sun_ang\n
        4 = pointsources\n
        5 = echans"""
    
    #sources_data = readfile.point_sources(readfile())
    sources_data = rf.point_sources()
    sources_names = sources_data[0]
    sources_coordinates = sources_data[1]
    sources_number = len(sources_names)
    
    user = getpass.getuser()
    
    if data_type == 'ctime':
        echans = np.arange(8)
    elif data_type == 'cspec':
        echans = np.arange(128)
    else:
        print 'Invalid data type: ' + str(data_type) + '\n Please insert an appropriate data type (ctime or cspec).'
        return
    
    cgb = []
    magnetic = []
    earth_ang = []
    sun_ang = []
    sources_coeff = []
    
    for echan in echans:
        print echan
    #See if Fit has already been calculated before
        if data_type == 'ctime':
            fitsname_data = 'ctime_' + detector_name + '_e' + str(echan) + '_kt.fits'
        else:
            fitsname_data = 'cspec_' + detector_name + '_e' + str(echan) + '__kt.fits'
    
        fits_path_data = '/home/' + user + '/Work/Fits_data/' + str(day) + '/'
        if not os.access(fits_path_data, os.F_OK):
            os.mkdir(fits_path_data)
        fitsfilepath_data = os.path.join(fits_path_data, fitsname_data)
        
        if os.path.isfile(fitsfilepath_data) == True:
            #data = readfile.fits_data (readfile(), day, detector_name, echan, data_type)
            data = rf.fits_data (day, detector_name, echan, data_type)
            fit_coeffs = data[10].astype(np.float)
            cgb1 = fit_coeffs[0]
            magnetic1 = fit_coeffs[1]
            earth_ang1 = fit_coeffs[2]
            sun_ang1 = fit_coeffs[3]
            sources_names = data[12]
            sources_coeff1 = data[13].astype(np.float)
            sources_coeff1 = sources_coeff1[sources_coeff1 != 0]
        else:
            print 'The file does not exist for the energy-channel ' + str(echan) + '. Fit the data first or review your input.'
            return
        
        cgb.append(cgb1)
        magnetic.append(magnetic1)
        earth_ang.append(earth_ang1)
        sun_ang.append(sun_ang1)
        sources_coeff.append(sources_coeff1)
        
    cgb = np.array(cgb)
    magnetic = np.array(magnetic)
    earth_ang = np.array(earth_ang)
    sun_ang = np.array(sun_ang)
    pointsources = [sources_names, map(list, zip(*sources_coeff))]
    
    return cgb, magnetic, earth_ang, sun_ang, pointsources, echans

In [29]:
#day = 150926
days = np.array([150926, 160913, 150914])
detectors = np.array(['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'na', 'nb'])
#detector = 'n5'
data_type = 'ctime'

cgb_all_days = []
magnetic_all_days = []
earth_all_days = []
sun_all_days = []
cyg_all_days = []
for day in days:
    print day
    cgb_all = []
    magnetic_all = []
    earth_all = []
    sun_all = []
    cyg_all = []
    for detector in detectors:
        print detector
        
        data = get_coeffs(day, detector)
        #print data
        cgb = data[0]
        magnetic = data[1]
        earth_ang = data[2]
        sun_ang = data[3]
        pointsources = data[4]
        echans = data[5]
        
        cyg = pointsources[1][-1]
        
        cgb_norm = cgb/cgb[2]
        magnetic_norm = magnetic/magnetic[4]
        earth_ang_norm = earth_ang/earth_ang[2]
        sun_ang_norm = sun_ang/sun_ang[1]
        cyg_norm = cyg/cyg[2]
        
        cgb_all.append(cgb_norm)
        magnetic_all.append(magnetic_norm)
        earth_all.append(earth_ang_norm)
        sun_all.append(sun_ang_norm)
        cyg_all.append(cyg_norm)
    
    cgb_all = np.array(cgb_all)
    magnetic_all = np.array(magnetic_all)
    earth_all = np.array(earth_all)
    sun_all = np.array(sun_all)
    cyg_all = np.array(cyg_all)
    
    cgb_all_days.append(cgb_all)
    magnetic_all_days.append(magnetic_all)
    earth_all_days.append(earth_all)
    sun_all_days.append(sun_all)
    cyg_all_days.append(cyg_all)

cgb_all_days = np.array(cgb_all_days)
magnetic_all_days = np.array(magnetic_all_days)
earth_all_days = np.array(earth_all_days)
sun_all_days = np.array(sun_all_days)
cyg_all_days = np.array(cyg_all_days)

150926
n0
0
1
2
3
4
5
6
7
n1
0
1
2
3
4
5
6
7
n2
0
1
2
3
4
5
6
7
n3
0
1
2
3
4
5
6
7
n4
0
1
2
3
4
5
6
7
n5
0
1
2
3
4
5
6
7
n6
0
1
2
3
4
5
6
7
n7
0
1
2
3
4
5
6
7
n8
0
1
2
3
4
5
6
7
n9
0
1
2
3
4
5
6
7
na
0
1
2
3
4
5
6
7
nb
0
1
2
3
4
5
6
7
160913
n0
0
1
2
3
4
5
6
7
n1
0
1
2
3
4
5
6
7
n2
0
1
2
3
4
5
6
7
n3
0
1
2
3
4
5
6
7
n4
0
1
2
3
4
5
6
7
n5
0
1
2
3
4
5
6
7
n6
0
1
2
3
4
5
6
7
n7
0
1
2
3
4
5
6
7
n8
0
1
2
3
4
5
6
7
n9
0
1
2
3
4
5
6
7
na
0
1
2
3
4
5
6
7
nb
0
1
2
3
4
5
6
7
150914
n0
0
1
2
3
4
5
6
7
n1
0
1
2
3
4
5
6
7
n2
0
1
2
3
4
5
6
7
n3
0
1
2
3
4
5
6
7
n4
0
1
2
3
4
5
6
7
n5
0
1
2
3
4
5
6
7
n6
0
1
2
3
4
5
6
7
n7
0
1
2
3
4
5
6
7
n8
0
1
2
3
4
5
6
7
n9
0
1
2
3
4
5
6
7
na
0
1
2
3
4
5
6
7
nb
0
1
2
3
4
5
6
7


In [30]:
chan_mid = np.array([10.76, 20.415, 38.803, 76.37, 190.19, 410.91, 751.94, 1466.43])
echans = np.array(['e0', 'e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'e7'])

In [92]:
### plot cgb ###
mpl.rcParams['axes.color_cycle'] = ['g', 'r', 'c', 'm', 'y', 'k']

index = np.array([0, 1, 3, 6, 7, 9])

for j in range(0, len(days)):
    cgb_all = cgb_all_days[j]
    cgb_mean = np.zeros(len(cgb_all[0]))
    for i in index:
        cgb_mean = np.sum([cgb_mean, cgb_all[i]], axis=0)
    
    cgb_mean = cgb_mean/len(index)
    plt.plot(chan_mid, cgb_mean, label = str(days[j]))

#for i in index:
#    cgb_all = cgb_all_days[0]
#    plt.plot(chan_mid, cgb_all[i], label = detectors[i])

In [ ]:
### plot magnetic ###
mpl.rcParams['axes.color_cycle'] = ['r', 'c', 'm', 'y', 'k']

index = np.array([2, 4, 5, 8, 10, 11])

for i in index:
    plt.plot(chan_mid, magnetic_all[i], label = detectors[i])

In [108]:
### plot earth ###
mpl.rcParams['axes.color_cycle'] = ['b', 'r', 'c', 'm', 'y', 'k']

#detectors = np.array(['n2', 'n4', 'n5', 'n8', 'na', 'nb'])
index = np.array([2, 4, 5, 8, 10, 11])

#for j in range(0, len(days)):
#    earth_all = earth_all_days[j]
#    earth_mean = np.zeros(len(earth_all[0]))
#    for i in index:
#        earth_mean = np.sum([earth_mean, earth_all[i]], axis=0)
#    
#    earth_mean = earth_mean/len(index)
#    plt.plot(chan_mid, earth_mean, label = str(days[j]))

for i in index:
    earth_all = earth_all_days[1]
    plt.plot(chan_mid, earth_all[i], label = detectors[i])

In [ ]:
### plot sun ###
mpl.rcParams['axes.color_cycle'] = ['r', 'c', 'm', 'y', 'k']

index = np.array([0, 1, 2, 3, 4, 5])

for i in index:
    plt.plot(chan_mid, sun_all[i], label = detectors[i])

In [ ]:
### plot cyg ###
mpl.rcParams['axes.color_cycle'] = ['r', 'c', 'm', 'y', 'k']



for i in range(0, len(cyg_all)):
    plt.plot(chan_mid, cyg_all[i], label = detectors[i])

In [ ]:
### Plot 1: Spectra

plt.plot(energies, intensity1, label = 'CGB', linewidth = 2)
plt.plot(energies[117:], earth_effect[117:], label = 'Effective Earth Occultation', linewidth = 2)
plt.plot(energies[:118], -earth_effect[:118], label = 'Negative effective Earth Occultation', linewidth = 2)


plt.xscale('log')
plt.yscale('log')
plt.legend(fontsize=30)
plt.grid()

plt.xlim(4, 2000)
plt.ylim(-1.5, 1.5)

plt.xlabel('Energy in keV', fontsize=40)
plt.ylabel('Intensity in [cm$^{-2}$ s$^{-1}$ keV$^{-1}$ sr$^{-1}$]', fontsize=40)
plt.tick_params(axis='both', which='major', labelsize=30)

plt.title('CGB intensity spectrum vs. the Earth albedo intesity spectrum', fontsize=40)

plt.show()

In [109]:
### Plot 2: Coeff spectra

energy_boundaries = np.array([[4, 12], [12, 27], [27, 50], [50, 102], [102, 295], [295, 540], [540, 985], [985, 2000]])

earth_effect = intensity3 - intensity1

cgb2_all = intensity1 * det_resp
earth2_all = earth_effect * det_resp

cgb2 = []
earth2 = []

for i in range(0, len(energy_boundaries)):
    bounds = energy_boundaries[i]
    cgb2.append(np.sum(cgb2_all[np.where(np.logical_and(energies >= bounds[0], energies < bounds[1]))]))
    earth2.append(np.sum(earth2_all[np.where(np.logical_and(energies >= bounds[0], energies < bounds[1]))]))

cgb2 = np.array(cgb2)
earth2 = np.array(earth2)


#plt.plot(energies, cgb_all/cgb_all[np.where(energies == 39)], label = 'CGB', linewidth = 2)
#plt.plot(energies, earth_all/earth_all[np.where(energies == 39)], label = 'Effective Earth Occultation', linewidth = 2)
#plt.plot(chan_mid, cgb2/cgb2[2], 'r-', label = 'CGB adjusted', linewidth = 10, alpha = 0.7)
#plt.plot(chan_mid, cgb2/cgb2[2], 'b-', label = 'CGB', linewidth = 10, alpha = 0.7)
plt.plot(chan_mid, earth2/earth2[2], 'g-', label = 'Effective Earth Occultation', linewidth = 10, alpha = 0.7)

plt.xscale('log')
#plt.yscale('log')
plt.legend(fontsize=30)
plt.grid()

plt.xlim(0, 1600)
plt.ylim(-.6, 2.6)

plt.xlabel('Energy in keV', fontsize=40)
#plt.ylabel('Coefficient value', fontsize=40)
plt.ylabel('Normalized coefficient value', fontsize=40)
plt.tick_params(axis='both', which='major', labelsize=30)

plt.title('CGB model coefficients vs. spectrum', fontsize=40)
#plt.title('Earth model coefficients vs. spectrum', fontsize=40)

plt.show()